In [1]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('plazas_prueba_2.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

                 CURP                       CCT    LATITUD   LONGITUD
0  GMIC250706MBSFLH83          Maria Luisa Ross  19.375208 -99.230397
1  VNQH790723MCHTPN78      Enrique de Olavarría  19.376023 -99.188159
2  GMIC250706MBSFLH83             Independencia  19.297861 -99.649142
3  VNQH790723MCHTPN78  Maestro Ernesto Alconedo  19.381043 -99.163434
4  GMIC250706MBSFLH83              Luis Cabrera  19.371972 -99.159487


In [2]:
import requests

# FUNCIÓN PARA HACER SOLICITUDES A LA API MAPBOX ISOCHRONES
def mapbox_isochrone(base_url, routing_profile, lon, lat, mins=None, meters=None, access_token=None):
    """
    Realiza una solicitud a la API Mapbox Isochrones y devuelve un GeoDataFrame con los vértices de la isócrona.
    """
    # Construye la URL de la solicitud
    if meters is not None:
        query = f"?contours_meters={meters}"
    elif mins is not None:
        query = f"?contours_minutes={mins}"
    else:
        raise ValueError("Debes especificar 'mins' o 'meters'.")
    
    url = f"{base_url}{routing_profile}/{lon},{lat}{query}&polygons=true&denoise=1&access_token={access_token}"

    # Realiza la solicitud a la API
    response = requests.get(url)
    response.raise_for_status()  # Lanza un error si la solicitud falla
    
    # Procesa los datos recibidos
    geom = response.json()["features"][0]["geometry"]["coordinates"][0]

    # Convierte las coordenadas a un DataFrame
    coords = pd.DataFrame(geom, columns=["lon", "lat"])
    return coords


# PRUEBA DE LA FUNCIÓN
test_coords = mapbox_isochrone(
    base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
    routing_profile="walking",
    lon=-99.184823289979, lat=19.33905855077223,
    meters=500,
    access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
)

test_coords

,lon,lat
0,-99.185823,19.341088
1,-99.186794,19.341029
2,-99.187358,19.339593
3,-99.188514,19.339059
4,-99.189093,19.338059
5,-99.187823,19.336529
6,-99.185823,19.335395
7,-99.184823,19.335892
8,-99.183823,19.335570
9,-99.182823,19.335835


In [15]:
import time
from shapely.geometry import Polygon

dataframe_ciclo = df  # Asignar el DataFrame que contiene los centros de trabajo

# CICLO PARA OBTENER ISÓCRONAS DE TODOS LOS CENTROS DE TRABAJO
isochrone_list = []  # Lista para guardar las isócronas
times = []  # Para estimar el tiempo de ejecución

for idx, row in dataframe_ciclo.iterrows():
    start_time = time.time()
    print(f"Obteniendo isócrona para la escuela {idx + 1} de {len(dataframe_ciclo)}...")

    try:
        # Obtener isócrona
        coords = mapbox_isochrone(
            base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
            routing_profile="driving-traffic",
            #routing_profile="walking",
            lon=row["LONGITUD"], lat=row["LATITUD"],
            #mins=5,  # Tiempo de conducción
            meters=5000,  # AQUI CAMBIAR LA DISTANCIA 5km 
            access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
        )
        # Convertir a polígono y añadir a la lista
        polygon = Polygon(coords.values)
        isochrone_list.append({"CCT": row["CCT"], "geometry": polygon})
    except Exception as e:
        print(f"Error al obtener la isócrona para la escuela {idx + 1}: {e}")
        continue

    # Calcular tiempo restante
    elapsed_time = time.time() - start_time
    times.append(elapsed_time)
    avg_time = sum(times) / len(times)
    remaining_time = avg_time * (len(dataframe_ciclo) - (idx + 1)) / 60
    print(f"Tiempo estimado restante: {remaining_time:.2f} minutos")

Obteniendo isócrona para la escuela 1 de 5...
Tiempo estimado restante: 0.03 minutos
Obteniendo isócrona para la escuela 2 de 5...
Tiempo estimado restante: 0.02 minutos
Obteniendo isócrona para la escuela 3 de 5...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 4 de 5...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 5 de 5...
Tiempo estimado restante: 0.00 minutos


In [16]:
import geopandas as gpd

# CONVERTIR A UN GEODATAFRAME
isochrones_gdf = gpd.GeoDataFrame(isochrone_list, crs="EPSG:4326")
#isochrones_gdf = gpd.GeoDataFrame(isochrone_list, crs="4326")

# IMPRIMIR BASE DE DATOS
isochrones_gdf.head()

,CCT,geometry
0,Maria Luisa Ross,"POLYGON ((-99.2154 19.39033, -99.21778 19.3885..."
1,Enrique de Olavarría,"POLYGON ((-99.19316 19.41509, -99.19416 19.415..."
2,Independencia,"POLYGON ((-99.65614 19.3303, -99.65714 19.3286..."
3,Maestro Ernesto Alconedo,"POLYGON ((-99.16443 19.41827, -99.16567 19.416..."
4,Luis Cabrera,"POLYGON ((-99.15349 19.41406, -99.15449 19.414..."


In [17]:
# UNIR RESULTADOS CON LA BASE ORIGINAL
escuelas_gdf = gpd.GeoDataFrame(dataframe_ciclo, geometry=None)
escuelas_gdf["geometry"] = None

for idx, row in escuelas_gdf.iterrows():
    match = isochrones_gdf[isochrones_gdf["CCT"] == row["CCT"]]
    if not match.empty:
        escuelas_gdf.at[idx, "geometry"] = match["geometry"].values[0]

# IMPRIMIR BASE DE DATOS
escuelas_gdf.head()

/tmp/ipykernel_17821/2735420822.py:3: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  escuelas_gdf["geometry"] = None


,CURP,CCT,LATITUD,LONGITUD,geometry
0,GMIC250706MBSFLH83,Maria Luisa Ross,19.375208,-99.230397,"POLYGON ((-99.2154 19.39033, -99.21778 19.3885..."
1,VNQH790723MCHTPN78,Enrique de Olavarría,19.376023,-99.188159,"POLYGON ((-99.19316 19.41509, -99.19416 19.415..."
2,GMIC250706MBSFLH83,Independencia,19.297861,-99.649142,"POLYGON ((-99.65614 19.3303, -99.65714 19.3286..."
3,VNQH790723MCHTPN78,Maestro Ernesto Alconedo,19.381043,-99.163434,"POLYGON ((-99.16443 19.41827, -99.16567 19.416..."
4,GMIC250706MBSFLH83,Luis Cabrera,19.371972,-99.159487,"POLYGON ((-99.15349 19.41406, -99.15449 19.414..."


In [13]:
#escuelas_gdf.explore()

In [18]:
import folium

# VISUALIZAR RESULTADOS CON FOLIUM
m = folium.Map(location=[19.33905855077223, -99.184823289979], zoom_start=12)

for _, row in escuelas_gdf.iterrows():
    if row["geometry"] is not None:
        folium.Polygon(
            locations=[(p[1], p[0]) for p in list(row["geometry"].exterior.coords)],
            color="blue",
            weight=1, fill_color="blue",
            fill_opacity=0.5,
            fill=True
        ).add_to(m)
        # Añadir un punto al mapa
        folium.Marker(
            location=[row['LATITUD'], row['LONGITUD']],  # Coordenadas del punto
            popup=f"CURP: {row['CURP']}, CCT: {row['CCT']}",  # Texto emergente al hacer clic en el marcador
            icon=folium.Icon(icon='info-sign')  # Icono del marcador
        ).add_to(m)

In [19]:
# Guardar resultados
isochrones_gdf.to_file("primarias.shp")
escuelas_gdf.to_file("escuelas_con_isocronas.shp")

/home/misael/.pyenv/versions/entorno_python_3.12.3/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [20]:
# Mostrar el mapa interactivo
m.save("isocronas_escuelas.html")

En esta parte se programa un codigo para extraer de forma eficiente todas los centros de trabajo de una misma persona.

In [22]:
# Asignar un identificador único a cada persona, en este caso la CURP
id = 'CURP' 

# Crear un diccionario para almacenar los DataFrames
dataframes = {}

# Agrupar por id y crear un DataFrame para cada grupo
for id_, grupo in escuelas_gdf.groupby(id):
    dataframes[id_] = grupo

# Mostrar los DataFrames creados
for id_, df_grupo in dataframes.items():
    print(f"DataFrame para {id_}: {df_grupo} \n")

DataFrame para GMIC250706MBSFLH83:                  CURP               CCT    LATITUD   LONGITUD  \
0  GMIC250706MBSFLH83  Maria Luisa Ross  19.375208 -99.230397   
2  GMIC250706MBSFLH83     Independencia  19.297861 -99.649142   
4  GMIC250706MBSFLH83      Luis Cabrera  19.371972 -99.159487   

                                            geometry  
0  POLYGON ((-99.2154 19.39033, -99.21778 19.3885...  
2  POLYGON ((-99.65614 19.3303, -99.65714 19.3286...  
4  POLYGON ((-99.15349 19.41406, -99.15449 19.414...   

DataFrame para VNQH790723MCHTPN78:                  CURP                       CCT    LATITUD   LONGITUD  \
1  VNQH790723MCHTPN78      Enrique de Olavarría  19.376023 -99.188159   
3  VNQH790723MCHTPN78  Maestro Ernesto Alconedo  19.381043 -99.163434   

                                            geometry  
1  POLYGON ((-99.19316 19.41509, -99.19416 19.415...  
3  POLYGON ((-99.16443 19.41827, -99.16567 19.416...   



In [25]:
from shapely.geometry import Point
from itertools import combinations

# Crear un DataFrame vacío con las columnas deseadas
df_out = pd.DataFrame(columns=[id, 'CCT 1', 'LONGITUD 1', 'LATITUD 1', 'geometry 1', 'CCT 2', 'LONGITUD 2', 'LATITUD 2', 'geometry 2'])

filas = []

# Mostrar los DataFrames creados
for name, df_group in dataframes.items():
    #print('Esto es ', df_group)
    for (idx, row), (idy, rowy) in combinations(df_group.iterrows(), 2):
        punto = Point(row['LONGITUD'], row['LATITUD'])
        poligono = rowy['geometry']
        print(f"Escuela del punto: {row['CCT']}")
        print(f"Escuela del poligono: {rowy['CCT']}")
        dentro = punto.within(poligono)
        print(f"dentro = {dentro}")
        if not dentro:
            filas.append({'CURP': row[id],
                           'CCT 1': row['CCT'],
                           'LONGITUD 1': row['LONGITUD'],
                           'LATITUD 1': row['LATITUD'],
                           'geometry 1': row['geometry'],
                           'CCT 2': rowy['CCT'],
                           'LONGITUD 2': rowy['LONGITUD'],
                           'LATITUD 2': rowy['LATITUD'],
                           'geometry 2': rowy['geometry'],
                           })
            
# Convertir la lista de filas en un DataFrame
df_out = pd.DataFrame(filas)

# Imprime la base el DataFrame resultante
df_out.head()

Escuela del punto: Maria Luisa Ross
Escuela del poligono: Independencia
dentro = False
Escuela del punto: Maria Luisa Ross
Escuela del poligono: Luis Cabrera
dentro = False
Escuela del punto: Independencia
Escuela del poligono: Luis Cabrera
dentro = False
Escuela del punto: Enrique de Olavarría
Escuela del poligono: Maestro Ernesto Alconedo
dentro = True


,CURP,CCT 1,LONGITUD 1,LATITUD 1,geometry 1,CCT 2,LONGITUD 2,LATITUD 2,geometry 2
0,GMIC250706MBSFLH83,Maria Luisa Ross,-99.230397,19.375208,"POLYGON ((-99.215397 19.390327, -99.217783 19....",Independencia,-99.649142,19.297861,"POLYGON ((-99.656142 19.330302, -99.657142 19...."
1,GMIC250706MBSFLH83,Maria Luisa Ross,-99.230397,19.375208,"POLYGON ((-99.215397 19.390327, -99.217783 19....",Luis Cabrera,-99.159487,19.371972,"POLYGON ((-99.153487 19.414064, -99.154487 19...."
2,GMIC250706MBSFLH83,Independencia,-99.649142,19.297861,"POLYGON ((-99.656142 19.330302, -99.657142 19....",Luis Cabrera,-99.159487,19.371972,"POLYGON ((-99.153487 19.414064, -99.154487 19...."


In [ ]:
from shapely.geometry import Point
from itertools import combinations

# Crear un DataFrame vacío con las columnas deseadas
df_out = pd.DataFrame(columns=[id, 'CCT 1', 'LONGITUD 1', 'LATITUD 1', 'geometry 1', 'CCT 2', 'LONGITUD 2', 'LATITUD 2', 'geometry 2'])

filas = []

# Mostrar los DataFrames creados
for name, df_group in dataframes.items():
    #print('Esto es ', df_group)
    for (idx, row), (idy, rowy) in combinations(df_group.iterrows(), 2):
        punto = Point(row['LONGITUD'], row['LATITUD'])
        poligono = rowy['geometry']
        print(f"Escuela del punto: {row['CCT']}")
        print(f"Escuela del poligono: {rowy['CCT']}")
        dentro = punto.within(poligono)
        print(f"dentro = {dentro}")
        if not dentro:
            filas.append({'CURP': row[id],
                           'CCT 1': row['CCT'],
                           'LONGITUD 1': row['LONGITUD'],
                           'LATITUD 1': row['LATITUD'],
                           'geometry 1': row['geometry'],
                           'CCT 2': rowy['CCT'],
                           'LONGITUD 2': rowy['LONGITUD'],
                           'LATITUD 2': rowy['LATITUD'],
                           'geometry 2': rowy['geometry'],
                           })
            
# Convertir la lista de filas en un DataFrame
df_out = pd.DataFrame(filas)

# Imprime la base el DataFrame resultante
df_out.head()

In [26]:
# Guardar el DataFrame df_out en un archivo CSV
df_out.to_csv('irregularidades.csv', index=False)